In [ ]:
import ee
import datetime
import pandas as pd
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Input directory containing all chunks
input_dir = 'Karnataka_Datasets/Across/Chunks'

# Output settings
export_folder = 'EarthEngine_VI_QA_CloudProb'
export_description_prefix = 'Karnataka_Chunk'

# Define dates
start_date = datetime.date(2021, 4, 1)
end_date = datetime.date(2022, 3, 31)
delta = datetime.timedelta(days=5)

dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += delta

# NDVI & GCVI function
def add_indices(image):
    image = ee.Image(image).multiply(0.0001)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    gcvi = image.select('B8').divide(image.select('B3')).subtract(1).rename('GCVI')
    return image.addBands([ndvi, gcvi])

# Cloud masking
def mask_clouds(image):
    qa60 = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    qa_mask = qa60.bitwiseAnd(cloud_bit_mask).eq(0).And(
              qa60.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    cloud_mask_img = ee.Image(image.get('cloud_mask'))
    
    return ee.Algorithms.If(
        cloud_mask_img,
        image.updateMask(qa_mask.And(cloud_mask_img.select('probability').lt(50))),
        image.updateMask(qa_mask)
    )

# Process each file in the input directory
for filename in os.listdir(input_dir):
    if not filename.endswith('.csv'):
        continue

    filepath = os.path.join(input_dir, filename)
    df = pd.read_csv(filepath)

    # Convert to FeatureCollection
    def row_to_feature(row):
        point = ee.Geometry.Point([row['Longitude'], row['Latitude']])
        return ee.Feature(point, {
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude'],
            'Crop_Name': row['Crop_Name']
        })

    features = [row_to_feature(row) for _, row in df.iterrows()]
    feature_collection = ee.FeatureCollection(features)

    all_fc = []

    for date in dates:
        start = ee.Date(str(date))
        end = start.advance(5, 'day')

        s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start, end) \
            .filterBounds(feature_collection) \
            .select(['B11', 'B12', 'B3', 'B4', 'B8', 'B8A', 'QA60'])

        s2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
            .filterDate(start, end) \
            .filterBounds(feature_collection)

        joined = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2_sr,
            secondary=s2_clouds,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )

        def process_image(img):
            img = ee.Image(img)
            masked = ee.Image(mask_clouds(img))
            return add_indices(masked)

        processed = ee.ImageCollection(joined).map(process_image)

        # Check if any image is available
        if processed.size().lte(0).getInfo():
            print(f"⚠️ No image found for {date}, skipping.")
            continue

        img = processed.median()

        reduced = img.reduceRegions(
            collection=feature_collection,
            reducer=ee.Reducer.mean(),
            scale=10
        )

        reduced = reduced.map(lambda f: f.set('Date', str(date)))
        all_fc.append(reduced)

    if all_fc:
        merged_fc = ee.FeatureCollection(all_fc).flatten()

        chunk_name = os.path.splitext(filename)[0]
        task = ee.batch.Export.table.toDrive(
            collection=merged_fc,
            description=f'{export_description_prefix}_{chunk_name}_QA_CloudProb',
            folder=export_folder,
            fileNamePrefix=f'{chunk_name}_QA_CloudProb',
            fileFormat='CSV'
        )
        task.start()
        print(f"✅ Export started for {chunk_name}")
    else:
        print(f"⚠️ No valid images found for any dates in {filename}, skipping export.")


⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_1
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_10
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_11
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_12
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_13
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_14
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_15
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_16
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image f